In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
import keras
import tensorflow as tf
from keras import metrics
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.metrics import AUC
from keras.optimizers import Adam,SGD
import os
import cv2

2024-04-27 09:12:43.298964: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 09:12:43.299085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 09:12:43.455814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
validation_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,)
val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

In [4]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size = (224, 224),
                                                    batch_size = 128, 
                                                    color_mode = 'rgb',
                                                    class_mode = 'binary', 
                                                    seed = 42)
val_generator = val_datagen.flow_from_directory(validation_dir, 
                                                 target_size = (224, 224),
                                                 batch_size = 128, 
                                                 class_mode = 'binary',
                                                 color_mode = 'rgb',
                                                 shuffle=False,
                                                 seed = 42)
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                 target_size = (224, 224),
                                                 batch_size = 128, 
                                                 class_mode = 'binary',
                                                 color_mode = 'rgb',
                                                 shuffle=False,
                                                 seed = 42)
test_generator.reset()

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001),  loss='binary_crossentropy',  metrics=[AUC(),'accuracy'])

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
history = model.fit(train_generator, 
                    steps_per_epoch = len(train_generator),
                    epochs = 40, 
                    validation_data = val_generator,
                    validation_steps = len(val_generator), 
                    verbose=1, batch_size=32)


Epoch 1/40


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-27 09:14:24.687972: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=1,k3=0} for conv (f32[128,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[128,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-04-27 09:14:24.827740: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.139866344s
Trying algo

39/41 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.8089 - auc: 0.8396 - loss: 0.4108

W0000 00:00:1714209529.291710     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


41/41 ━━━━━━━━━━━━━━━━━━━━ 325s 5s/step - accuracy: 0.8126 - auc: 0.8457 - loss: 0.4036 - val_accuracy: 0.7500 - val_auc: 0.9219 - val_loss: 0.5927
Epoch 2/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/40


W0000 00:00:1714209547.497591     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


41/41 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.9249 - auc: 0.9704 - loss: 0.1960 - val_accuracy: 0.8125 - val_auc: 0.9531 - val_loss: 0.3932
Epoch 4/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.9359 - auc: 0.9781 - loss: 0.1645 - val_accuracy: 0.8125 - val_auc: 0.9688 - val_loss: 0.3396
Epoch 6/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.9485 - auc: 0.9833 - loss: 0.1435 - val_accuracy: 0.9375 - val_auc: 0.9844 - val_loss: 0.2408
Epoch 8/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val

In [8]:
model.evaluate(test_generator)

W0000 00:00:1714212299.992241     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/5 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8001 - auc: 0.7213 - loss: 0.6752   

2024-04-27 10:05:17.887643: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=1,k3=0} for conv (f32[112,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[112,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-04-27 10:05:17.939887: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.052426096s
Trying algorithm eng11{k2=1,k3=0} for conv (f32[112,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[112,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0

5/5 ━━━━━━━━━━━━━━━━━━━━ 98s 24s/step - accuracy: 0.8299 - auc: 0.7993 - loss: 0.5746


[0.37323370575904846, 0.9553583860397339, 0.8894230723381042]

In [15]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

# Load the ResNet50 base model without the top layer
base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base.layers:
    layer.trainable = False

# Create the top layers for the model
x = Flatten()(base.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Combine the base model with the top layers
model1 = Model(inputs=base.input, outputs=output)

# Compile the model
model1.compile(optimizer=Adam(learning_rate=0.0001),  
              loss='binary_crossentropy',  
              metrics=[AUC(),'accuracy'])

# Train the model
history1 = model1.fit(train_generator, 
                    steps_per_epoch=len(train_generator),
                    epochs=40, 
                    validation_data=val_generator,
                    validation_steps=len(val_generator), 
                    verbose=1, 
                    batch_size=32)


Epoch 1/40


W0000 00:00:1714213314.169406     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


36/41 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7397 - auc_3: 0.6483 - loss: 0.5659

W0000 00:00:1714213407.637966     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


41/41 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - accuracy: 0.7414 - auc_3: 0.6643 - loss: 0.5572 - val_accuracy: 0.6250 - val_auc_3: 0.8281 - val_loss: 0.6350
Epoch 2/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - auc_3: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc_3: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.7986 - auc_3: 0.8583 - loss: 0.4200 - val_accuracy: 0.5625 - val_auc_3: 0.8438 - val_loss: 0.8489
Epoch 4/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - accuracy: 0.0000e+00 - auc_3: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc_3: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.8269 - auc_3: 0.8887 - loss: 0.3722 - val_accuracy: 0.5625 - val_auc_3: 0.8359 - val_loss: 1.0386
Epoch 6/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - accuracy: 0.0000e+00 - auc_3: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc

In [16]:
model1.evaluate(test_generator)

W0000 00:00:1714215407.856322     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 704ms/step - accuracy: 0.8807 - auc_3: 0.7113 - loss: 0.3407

W0000 00:00:1714215430.114919     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5/5 ━━━━━━━━━━━━━━━━━━━━ 26s 6s/step - accuracy: 0.8804 - auc_3: 0.7886 - loss: 0.3291 


[0.3059775233268738, 0.9430637955665588, 0.879807710647583]

In [18]:
import numpy as np
test_data, true_labels = next(test_generator)  
predictions_model = model.predict(test_data)
predictions_model1 = model1.predict(test_data)
ensemble_predictions = (predictions_model + predictions_model1) / 2
ensemble_classes = np.round(ensemble_predictions)
ensemble_accuracy = np.mean(ensemble_classes == true_labels)
print("Ensemble Accuracy:", ensemble_accuracy)


2/4 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step

W0000 00:00:1714216000.157188     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 133ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step
Ensemble Accuracy: 0.78125


In [19]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

# Load InceptionV3 base model without the top layer
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in inception.layers:
    layer.trainable = False

# Create the top layers for the model
x = Flatten()(inception.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Combine the base model with the top layers
model_inception = Model(inputs=inception.input, outputs=output)

# Compile the model
model_inception.compile(optimizer=Adam(learning_rate=0.0001),  
                        loss='binary_crossentropy',  
                        metrics=[AUC(),'accuracy'])

# Train the model
history_inception = model_inception.fit(train_generator, 
                                        steps_per_epoch=len(train_generator),
                                        epochs=40, 
                                        validation_data=val_generator,
                                        validation_steps=len(val_generator), 
                                        verbose=1, 
                                        batch_size=32)


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/40


W0000 00:00:1714216818.776004     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


30/41 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.7219 - auc_4: 0.7392 - loss: 0.5060

W0000 00:00:1714216911.030110     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


41/41 ━━━━━━━━━━━━━━━━━━━━ 175s 3s/step - accuracy: 0.7565 - auc_4: 0.7863 - loss: 0.4548 - val_accuracy: 0.9375 - val_auc_4: 0.9688 - val_loss: 0.2829
Epoch 2/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - auc_4: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc_4: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.9307 - auc_4: 0.9769 - loss: 0.1683 - val_accuracy: 0.9375 - val_auc_4: 0.9844 - val_loss: 0.2259
Epoch 4/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.0000e+00 - auc_4: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc_4: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.9393 - auc_4: 0.9806 - loss: 0.1576 - val_accuracy: 0.9375 - val_auc_4: 0.9688 - val_loss: 0.2799
Epoch 6/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.0000e+00 - auc_4: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_auc

In [20]:
model_inception.evaluate(test_generator)

W0000 00:00:1714218858.092407     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 661ms/step - accuracy: 0.7344 - auc_4: 0.7142 - loss: 0.7379

W0000 00:00:1714218890.002631     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 8s/step - accuracy: 0.7754 - auc_4: 0.7917 - loss: 0.6269 


[0.404865562915802, 0.9467291235923767, 0.8573718070983887]

In [21]:
import numpy as np

predictions_vgg19 = model.predict(test_generator)
predictions_resnet50 = model1.predict(test_generator)
predictions_inception = model_inception.predict(test_generator)

ensemble_predictions = (predictions_vgg19 + predictions_resnet50 + predictions_inception) / 3

ensemble_classes = np.round(ensemble_predictions)

ensemble_accuracy = np.mean(ensemble_classes == true_labels)

print("Ensemble Accuracy:", ensemble_accuracy)


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 809ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 808ms/step
1/5 ━━━━━━━━━━━━━━━━━━━━ 30s 8s/step

W0000 00:00:1714219006.137621     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step 
Ensemble Accuracy: 0.29967948717948717


W0000 00:00:1714219013.972771     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [22]:
import numpy as np

predictions_resnet50 = model1.predict(test_generator)
predictions_inception = model_inception.predict(test_generator)

ensemble_predictions = (predictions_resnet50 + predictions_inception) / 2

ensemble_classes = np.round(ensemble_predictions)

ensemble_accuracy = np.mean(ensemble_classes == true_labels)

print("Ensemble Accuracy:", ensemble_accuracy)


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 803ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 808ms/step
Ensemble Accuracy: 0.3060897435897436


In [23]:
import numpy as np

predictions_vgg19 = model.predict(test_generator)
predictions_inception = model_inception.predict(test_generator)

ensemble_predictions = (predictions_vgg19 +predictions_inception) / 2

ensemble_classes = np.round(ensemble_predictions)

ensemble_accuracy = np.mean(ensemble_classes == true_labels)

print("Ensemble Accuracy:", ensemble_accuracy)


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 802ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 821ms/step
Ensemble Accuracy: 0.27724358974358976


In [24]:
import numpy as np

predictions_vgg19 = model.predict(test_generator)
predictions_resnet50 = model1.predict(test_generator)
predictions_inception = model_inception.predict(test_generator)

ensemble_predictions = (predictions_vgg19 + predictions_resnet50 + predictions_inception) / 3

ensemble_classes = np.round(ensemble_predictions)

ensemble_accuracy = np.mean(ensemble_classes == true_labels)

print("Ensemble Accuracy:", ensemble_accuracy)


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 882ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 831ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 898ms/step
Ensemble Accuracy: 0.29967948717948717
